# COGS 108 - Final Project 

# Team 178: Small-Baller Brand

- George Hu - A13679845
- Ryan Shim - A12820194
- Sonny Zhou - A13080043

# Overview

The NBA has been experiencing a transition from a half court pacing to running and spreading the floor by using more shooters in lineups. We want to investigate whether or not there has been a statistically significant change in the average height and weight of lineups/players in the NBA over the years (transition to small-ball, a strategy modern NBA teams employ to run faster paced plays with better ball movement). We then want to explore if there is actually a correlation between the height and weight of NBA players and their shooting percentages (in terms of 3-pointers and free throws) in order to investigate whether or not bigger players are changing their playstyles. Furthermore, we want to explore the change in the strategies employed by NBA teams based on where they shoot the ball and the value of the shots taken.

Has the NBA started incorporating more small-ball related strategies over the past few decades? Our group wants to find out if there has been a recent meta shift in the NBA towards using small-ball strategies. According to Wikipedia, small-ball is defined as “a style of play that sacrifices height, physical strength and low post offense/defense in favor of a lineup of smaller players for speed, agility and increased scoring (often from the three-point line).” We'll use various visualizations to see how the NBA meta has changed in terms of height, weight, 3-point percentages, and other factors.


# Research Question

Has the NBA transitioned towards employing more “small-ball” strategies over the past few years/decades? If so, is a small-ball strategy effective for winning games? 


## Background and Prior Work

https://towardsdatascience.com/the-evolution-of-the-nba-3-point-line-da6700714ad2 
<br>
The above article is a study done regarding the evolution of the 3 point shot over time. As the data indicates, not only is there a greater number of 3 point shots taken, but there is also a proportionally greater number of 3 point shots when compared to 2 point shots. In fact, while the number of 2 point shots taken have been stagnant for the last 40 years, it seems that the number of 3 point shots taken have always increased every 5 years. Furthermore, there seems to be extreme outliers in terms of players in regards to the 3 point shot, as it can be seen from that data that players like Steph Curry and James Harden are shooting more 3 pointers than ever before in history. The above factors seem to indicate that there is indeed a greater emphasis in spreading the floor when compared to before, as there is a greater need for players to get adequate room to shoot from afar. Furthermore, it would seem to indicate that smaller players who are better shooters do seem to be favored in the modern NBA court, as they seem to be becoming the focus of the team’s offense when compared to prior eras.

http://harvardsportsanalysis.org/2017/11/maybe-we-should-call-it-skinnyball-weight-vs-height-in-the-nba/
<br>
This article describes the small-ball phenomenon as it attempts to find statistically significant data that supports the claim that NBA teams have shifted their strategy to fit around the meta changes of smaller players. The data shows that while the NBA has drastically increased the height and weight of players since the 1960’s, in recent years, the weighted weight of the league has dropped nearly 5 pounds. While the article did not find any statistically significant data regarding lighter or shorter teams winning a higher number of games, it is undeniable that most star players such as Kevin Durant, Chris Bosh and Kevin Love are much skinnier than previous renditions of the role. Furthermore, the data shows that lighter teams have produced a faster playstyle, and using their natural athletic abilities, new superstars are able to out run and out pace heavier teams.


# Hypothesis


Our group, Small Baller Brand, hypothesizes that small-ball strategies have been more effective in recent years, especially with the NBA meta changing to more high quality point guards and a decrease in the number of big men.

We chose this hypothesis because we wanted to see which direction the basketball league will be trending to in the future. Anecdotally, we also noticed that players such as Steph Curry have chosen to shoot three pointers over other forms of scoring and have seen an increased win-rate because of it. We hope to find a positive correlation between small-ball strategies and win rate



# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: NBA Players
- Link to the dataset: https://www.kaggle.com/drgilermo/nba-players-stats
- Number of observations: 3922 rows x 8 columns for players

1-2 sentences describing each dataset. 

This dataset records all the players that have played in the NBA from the 1950 to 2017 seasons.

- Dataset Name: NBA Seasonal Player stats
- Link to the dataset: https://www.kaggle.com/drgilermo/nba-players-stats
- Number of observations: 24691 rows × 53 columns for seasons

1-2 sentences describing each dataset. 

This dataset records all the players' seasonal stats per year starting from 1950 to 2017.

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

We intend to use the player weight/height columns from the players dataframe and merge it into the seasonal stats dataframe in order to be able to access weights/heights of the entire NBA per season.

# Setup

In [10]:
## YOUR CODE HERE
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation

# Data Cleaning

Describe your data cleaning steps here.

We took our data from https://www.kaggle.com/drgilermo/nba-players-stats and combined the two CSVs in player_data.csv and Season_Stats.csv in order to make our dataframe.

In [25]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION
players = pd.read_csv('Players.csv')
seasons = pd.read_csv('Seasons_Stats.csv')

# These lines drops all the rows of players and seasons where the Player name is null
players = players[pd.isnull(players.Player) == 0]
seasons = seasons[pd.isnull(seasons.Player) == 0]

# These lines add the player's heights and weights to the seasonal data
seasons['height'] = seasons.Player.apply(lambda x: players.height[players.Player == x].values[0])
seasons['weight'] = seasons.Player.apply(lambda x: players.weight[players.Player == x].values[0])

In [26]:
# This is in a separate cell so that seasons don't have to be loaded multiple times
print(seasons['height'])
print(seasons['weight'])

0        180.0
1        188.0
2        193.0
3        196.0
4        196.0
5        196.0
6        178.0
7        180.0
8        196.0
9        196.0
10       196.0
11       183.0
12       196.0
13       196.0
14       193.0
15       208.0
16       190.0
17       196.0
18       196.0
19       185.0
20       193.0
21       193.0
22       208.0
23       196.0
24       190.0
25       190.0
26       190.0
27       183.0
28       196.0
29       193.0
         ...  
24661    190.0
24662    190.0
24663    190.0
24664    190.0
24665    190.0
24666    190.0
24667    206.0
24668    206.0
24669    206.0
24670    185.0
24671    201.0
24672    198.0
24673    198.0
24674    198.0
24675    208.0
24676    201.0
24677    213.0
24678    211.0
24679    190.0
24680    211.0
24681    196.0
24682    198.0
24683    188.0
24684    201.0
24685    203.0
24686    213.0
24687    213.0
24688    213.0
24689    203.0
24690    216.0
Name: height, Length: 24624, dtype: float64
0         77.0
1         83.0
2         8

# Data Analysis & Results

Include cells that describe the steps in your data analysis.

In [5]:
## YOUR CODE HERE
## FEEL FREE TO ADD MULTIPLE CELLS PER SECTION

# Ethics & Privacy

Our group will find and use public datasets to ensure full transparency of our dataset and where they come from. Also, our team will not bash players, but rather speak only of the numbers and interpret the numbers and statistics in an appropriate context. Our aim is to not hold any team as “worse” than any other team, unless the numbers say so otherwise. This means our team doesn’t need to ask players for their data or do any “spying” on players. Our end goal is to highlight whether small-ball strategy is effective or not, and not to bash on any other strategy that a player/team may employ. No offence will be given or issued from our data analysis. The purpose of this project is purely for educational use, and will not benefit any third party outside of school boundaries.

Our data source will be obtained from Kaggle and https://www.basketball-reference.com/;  we assume that all the data collected is unbiased since basketball-reference.com simply tracks the numbers that occurred during a match and aggregates this data into other statistics, like 3 pt %. We suspect no harm of bias in this data because there shouldn’t be any emotional attachments to it (eg. no one is placing bets on these data and affecting the NBA players themselves). Our Kaggle dataset also comes from Omri Goldstein, a top data scientist from Israel who has participated in many data science competitions and is a leader in the data sciences. He has competed in numerous data science competitions and we trust that the judges in all of these competitions have vetted him for data biases. Kaggle is also a source for high quality datasets that many data scientists use to find sources of data. Furthermore, as the numbers we’ll be using are empirical data such as the player’s height, weight, etc., we don’t believe there is generally a room for error or emotional biases within our datasets. However, there are cases such as Kevin Durant, whose real height is supposedly much taller than his recorded height, so that could cause room for error even in the empirical data we’re tracking.

Through this research, we simply hope to verify the existence of small-ball strategies and whether even big centers are forced to spread the floor by being able to shoot 3 point shots. Therefore, we don’t believe that there will be any negative consequences to our research and unbiasedly hope to confirm or deny people’s belief in small-ball strategies over time. Lastly, we don’t believe that there will be any hugely impactful unintended consequences because we’re focusing on an idea that’s already present in most people’s minds, but not verified through hard data.


# Conclusion & Discussion

*Fill in your discussion information here*